In [8]:
import math
from web3 import Web3, AsyncWeb3
from web3 import eth
import abi as abi_import
import time
import datetime
from datetime import timedelta, date, datetime, time
import requests

In [34]:
curr = datetime.utcnow()
ts = str(curr.year) + '-' + str(curr.month) + '-' + str(curr.day) + " " + str(curr.hour) + ":00:00"

delta = timedelta(days=10)

curr_minusTen = datetime.utcnow() - delta

ts_back = str(curr_minusTen.year) + '-' + str(curr_minusTen.month) + '-' + str(curr_minusTen.day) + " " + str(curr_minusTen.hour) + ":00:00"
print(ts)
print(ts_back)

print(time(curr_minusTen.hour, curr_minusTen.minute, curr_minusTen.second, 0))

2023-10-20 10:00:00
2023-10-10 10:00:00
10:50:02


In [60]:
addr = '0xc9becdbc62efb867cb52222b34c187fb170379c6'

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

# s = hour_rounder(curr).isoformat() + ".000Z"
# e = hour_rounder(curr_minusTen).isoformat() + ".000Z"

s =  datetime.now()
e = datetime.now() - timedelta(days=1)

s = s.replace(hour=0,minute=0,second=0,microsecond=0)
e = e.replace(hour=0,minute=0,second=0,microsecond=0)


payload = {'time_frame': 'hour', 'timestamp_start': s, 'timestamp_end': e}

u = 'https://api-v2.pendle.finance/core/v2/1/markets/' + addr + '/history'
print(u)

u1 = 'https://api-v2.pendle.finance/core/v2/1/markets/0xc9becdbc62efb867cb52222b34c187fb170379c6/history?time_frame=hour&timestamp_start=2023-10-20&timestamp_end=2023-10-15'


r = requests.get(u)
res = r.json()['results'] # timestamp,  'underlyingInterestApy', 'underlyingRewardApy', 'underlyingApy',
                        # 'impliedApy','ytFloatingApy' ,'ptDiscount'


ts_lst = []
underlyingInterestAPY_lst = []
underlyingRewardAPY_lst = []
underlyingAPY_lst = []
impliedAPY_lst = []
longYieldAPY = []
ptPrice = []
ytPrice = []

print(len(res))

for i in range(0, len(res)):
    ts_lst.append(res[i]['timestamp'])
    underlyingInterestAPY_lst.append(res[i]['underlyingInterestApy'])
    underlyingRewardAPY_lst.append(res[i]['underlyingRewardApy'])
    underlyingAPY_lst.append(res[i]['underlyingApy'])
    impliedAPY_lst.append(res[i]['impliedApy'])
    longYieldAPY.append(res[i]['ytFloatingApy'])
    ytPrice.append(res[i]['ptDiscount'])
    ptPrice.append(1.0 - res[i]['ptDiscount'])




https://api-v2.pendle.finance/core/v2/1/markets/0xc9becdbc62efb867cb52222b34c187fb170379c6/history
225


In [69]:
import pickle

with open('pendleData.pkl', 'rb') as fp:
    tmpDict = pickle.load(fp)

util_ts = []
crvUtil = []

for j in tmpDict.keys():
    util_ts.append(datetime.fromtimestamp(tmpDict[j][7]))
    crvUtil.append(tmpDict[j][5])



In [ ]:
print()